In [174]:
# futures_grid_bot_sim.py
# Full simulation pipeline — uses live data from Binance via ccxt, but DOES NOT place real trades.
# It will open LONG grids when model predicts uptrend and SHORT grids when model predicts downtrend.
# No leverage is used (placeholder comments show where to add leverage later).

import ccxt
import torch
import torch.nn as nn
import joblib
import numpy as np
import pandas as pd
import ta
import talib
import uuid
import time
from datetime import datetime, timedelta, timezone
import math


In [175]:
import ccxt

# === Connect to Binance Futures Testnet ===
exchange = ccxt.binance({
    'apiKey': 'q0HAhhxXOJr8HBbmYFYe0QMhPMrbfWVGngOUeccqXQHdgCk5fKqq97vfykJm8EGk',
    'secret': 'LSkB2x2lO8r5fx6qaTR4rGObGFjRHbJHwHPbpSY4LQ8Hp9uU7vKaueVaDavbVA3C',
    'options': {'defaultType': 'future'},
})

exchange.set_sandbox_mode(True)  # Enable Testnet mode

print("✅ Connected to Binance Futures Testnet")





✅ Connected to Binance Futures Testnet


In [176]:
# -------------------------
# CONFIG
# -------------------------
symbol = "BTC/USDT"        # we fetch futures-market data (via ccxt options)
model_timeframe = "15m"
price_timeframe = "1s"
limit = 300
window_size = 10

FEATURES = [
    'rsi', 'ema12', 'ema26', 'macd', 'signal', 'histogram',
    'dema9', 'sma3', 'tsi', '%k', '%d',
    'atr', 'obv', 'adx', 'cci',
    'return_1', 'roll_mean_5', 'roll_std_5'
]

SAVED_MODEL = "greg_tech_8.pth"
SCALER_FILE = "scaler_15m.pkl"

min_tp_pct = 0.01
atr_period = 14
atr_multiplier = 0.1
position_fraction = 0.05
reduced_fraction = 0.5
grid_levels = 16
grid_spacing_atr_factor = 0.1
max_total_exposure = 0.7

prediction_interval = 15 * 60
check_interval_seconds = 1

hold_factor = 0.2

# Forced cutoff parameters for frozen grids
frozen_timeout_seconds = 60 * 60
frozen_max_drawdown_pct = 0.2

# device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [177]:
# -------------------------
# MODEL (your LSTM)
# -------------------------
class CryptoLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(CryptoLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, (hn, cn) = self.lstm(x)
        out = out[:, -1, :]
        out = self.fc(out)
        return out

input_dim = len(FEATURES)
hidden_dim = 128
num_layers = 4
output_dim = 3

model = CryptoLSTM(input_dim, hidden_dim, num_layers, output_dim).to(device)
model.load_state_dict(torch.load(SAVED_MODEL, map_location=device))
model.eval()
scaler = joblib.load(SCALER_FILE)

/home/gregory/Trading_bot/venv/lib/python3.12/site-packages/sklearn/base.py:442: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.6.1 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [178]:
# Helper: fetch minimums / step sizes for rounding
MARKET_INFO = {}
def load_market_info():
    global MARKET_INFO
    markets = exchange.load_markets()
    if symbol in markets:
        info = markets[symbol]
        MARKET_INFO['precision'] = info.get('precision', {})
        MARKET_INFO['limits'] = info.get('limits', {})
    else:
        MARKET_INFO['precision'] = {}
        MARKET_INFO['limits'] = {}
load_market_info()

In [179]:
# -------------------------
# INDICATORS (same compute_indicators but lowercase columns)
# -------------------------
def compute_indicators(df):
    df = df.copy()
    df.columns = [c.lower() for c in df.columns]

    out = {}
    # Core
    out["RSI"] = ta.momentum.RSIIndicator(df["close"], window=14).rsi()
    out["EMA12"] = df["close"].ewm(span=12, adjust=False).mean()
    out["EMA26"] = df["close"].ewm(span=26, adjust=False).mean()
    out["MACD"] = out["EMA12"] - out["EMA26"]
    out["Signal"] = out["MACD"].ewm(span=9, adjust=False).mean()
    out["Histogram"] = out["MACD"] - out["Signal"]
    # DEMA
    try:
        out["DEMA9"] = talib.DEMA(df["close"].values, timeperiod=9)
    except Exception:
        out["DEMA9"] = df["close"].ewm(span=9, adjust=False).mean()
    # SMA
    out["SMA3"] = ta.trend.sma_indicator(df["close"], window=3)
    # TSI
    def compute_tsi(close, r1=25, r2=13):
        delta = close.diff()
        ema1 = delta.ewm(span=r1, adjust=False).mean()
        ema2 = ema1.ewm(span=r2, adjust=False).mean()
        abs_delta = delta.abs()
        abs_ema1 = abs_delta.ewm(span=r1, adjust=False).mean()
        abs_ema2 = abs_ema1.ewm(span=r2, adjust=False).mean()
        tsi = 100 * (ema2 / (abs_ema2.replace(0, np.nan)))
        return tsi.fillna(0)
    out["TSI"] = compute_tsi(df["close"])
    # Stochastic
    period, smooth_k, smooth_d = 14, 3, 3
    lowest_low = df["low"].rolling(period).min()
    highest_high = df["high"].rolling(period).max()
    out["%K"] = 100 * (df["close"] - lowest_low) / (highest_high - lowest_low + 1e-8)
    out["%K"] = out["%K"].rolling(smooth_k).mean()
    out["%D"] = out["%K"].rolling(smooth_d).mean()
    # Advanced
    out["ATR"] = ta.volatility.AverageTrueRange(df["high"], df["low"], df["close"], window=14).average_true_range()
    out["OBV"] = ta.volume.OnBalanceVolumeIndicator(df["close"], df["volume"]).on_balance_volume()
    out["ADX"] = ta.trend.ADXIndicator(df["high"], df["low"], df["close"], window=14).adx()
    out["CCI"] = ta.trend.cci(df["high"], df["low"], df["close"], window=20)
    # Price derived
    out["Return_1"] = df["close"].pct_change()
    out["Roll_Mean_5"] = df["close"].rolling(5).mean()
    out["Roll_Std_5"] = df["close"].rolling(5).std()

    feat_df = pd.DataFrame(out, index=df.index)
    feat_df = feat_df.dropna().reset_index(drop=True)
    feat_df.columns = [c.lower() for c in feat_df.columns]
    return feat_df

In [180]:
# -------------------------
# FETCH DATA
# -------------------------
def fetch_latest_ohlcv(symbol, timeframe, limit):
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)
    df = pd.DataFrame(ohlcv, columns=['timestamp','open','high','low','close','volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df.columns = [c.lower() for c in df.columns]
    return df

def get_current_atr():
    df = fetch_latest_ohlcv(symbol, model_timeframe, atr_period + 5)
    df['prev_close'] = df['close'].shift(1)
    df['TR'] = df.apply(lambda r: max(r['high'] - r['low'],
                                      abs(r['high'] - r['prev_close']),
                                      abs(r['low'] - r['prev_close'])), axis=1)
    df['ATR'] = df['TR'].rolling(atr_period).mean()
    return float(df['ATR'].iloc[-1])


In [181]:
# -------------------------
# CCXT EXCHANGE SETUP
# -------------------------
exchange_2 = ccxt.binance({
    'enableRateLimit': True,
})

In [182]:
def fetch_latest_ohlcv_2(symbol, timeframe, limit):
    ohlcv = exchange_2.fetch_ohlcv(symbol, timeframe=timeframe, limit=limit)
    df = pd.DataFrame(ohlcv, columns=['timestamp','open','high','low','close','volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df.columns = [c.lower() for c in df.columns]
    return df

In [183]:
# -------------------------
# Prediction
# -------------------------
def predict_label_from_window(window_features):
    # window_features: numpy array shape (window_size, features)
    flat = scaler.transform(window_features)
    x = torch.tensor(flat, dtype=torch.float32).unsqueeze(0).to(device)
    with torch.no_grad():
        logits = model(x)
        probs = torch.softmax(logits, dim=1).cpu().numpy()[0]
        probs[1] *= hold_factor
        probs = probs / probs.sum()
        pred = int(np.argmax(probs))
    return pred  # 0=down 1=hold 2=up


In [184]:
# -------------------------
# Trading state & helpers
# -------------------------
balance = exchange.fetch_balance()
open_positions = exchange.fetch_positions()   # each: {id, side: 'long'/'short', entry, qty, notional(size_usd), grid_id, created_at, tp_price, sl_price, last_price}
open_orders = exchange.fetch_open_orders(symbol)      # each: {id, grid_id, side, price, size_usd, status, ccxt_id, tp, sl}
grids = {}            # grid metadata


def round_amount_qty(qty):
    prec = MARKET_INFO.get('precision', {}).get('amount')
    if prec is None:
        return qty
    prec = int(prec)
    return float(round(qty, prec))

def round_price(price):
    prec = MARKET_INFO.get('precision', {}).get('price')
    if prec is None:
        return price
    prec = int(prec)
    return float(round(price, prec))

def usd_to_qty(usd, price):
    return usd / price



In [185]:
# -------------------------
# REAL FUTURES order helpers (Binance only)
# -------------------------

def place_limit_order_ccxt(side, price, size_usd, symbol=symbol):
    """
    Place a real futures limit order on Binance.
    side: 'buy' (long) or 'sell' (short)
    """
    price = float(price)
    amount = size_usd / price
    amount = round_amount_qty(amount)
    price = round_price(price)
    if amount <= 0:
        raise ValueError("Amount <= 0 after rounding; increase size_usd or check precision.")

    params = {
        'timeInForce': 'GTC',
        'positionSide': 'LONG' if side.lower() == 'buy' else 'SHORT'
    }

    try:
        order = exchange.create_order(symbol, 'limit', side, amount, price, params)
        return order
    except Exception as e:
        print("[CCXT PLACE LIMIT ERROR]", e)
        return None


def fetch_order_ccxt(ccxt_order_id, symbol=symbol):
    """
    Fetch a real futures order by its CCXT order id.
    """
    try:
        return exchange.fetch_order(ccxt_order_id, symbol)
    except Exception as e:
        print("[CCXT FETCH ORDER ERROR]", e)
        return None


def cancel_order_ccxt(ccxt_order_id, symbol=symbol):
    """
    Cancel a real futures order by its CCXT order id.
    """
    try:
        exchange.cancel_order(ccxt_order_id, symbol)
        return True
    except Exception as e:
        print("[CCXT CANCEL ORDER ERROR]", e)
        return False


def market_close_position_ccxt(pos):
    """
    Close an open futures position on Binance.
    pos: {'side', 'qty', 'symbol'}
    """
    try:
        amount = round_amount_qty(pos['qty'])
        close_side = 'sell' if pos['side'].lower() == 'long' else 'buy'
        params = {'reduceOnly': True, 'positionSide': pos['side'].upper()}
        order = exchange.create_order(pos['symbol'], 'market', close_side, amount, None, params)
        return order
    except Exception as e:
        print("[CCXT MARKET CLOSE ERROR]", e)
        return None


In [186]:
def recreate_grid_order(price, side):
    """
    Place a futures limit order on Binance (testnet) based on the grid logic.
    """
    global open_orders, grids, balance

    # Find a grid that matches this price (approx)
    grid_id = None
    for gid, meta in grids.items():
        for oid in meta.get('orders', []):
            for o in open_orders:
                if abs(o['price'] - price) < 1e-8 and o['grid_id'] == gid:
                    grid_id = gid
                    break

    if grid_id is None and len(grids) > 0:
        grid_id = list(grids.keys())[0]

    # Calculate ATR-based spacing and order size
    atr_now = get_current_atr()
    spacing = atr_now * grid_spacing_atr_factor
    size_usd = max(20.0, balance * position_fraction)  # balance in USDT

    # Determine order side for Binance
    binance_side = 'BUY' if side == 'long' else 'SELL'

    # Take Profit and Stop Loss prices
    if side == 'long':
        tp = price + spacing
        sl = price - 0.5 * spacing
    else:
        tp = price - spacing
        sl = price + 0.5 * spacing

    # Calculate order amount in contracts
    amount = round_amount_qty(size_usd / price)

    # Place real Binance limit order
    params = {
        'type': 'LIMIT',
        'timeInForce': 'GTC',
        # You can set positionSide here if using dual-side mode
        # 'positionSide': 'LONG' or 'SHORT'
    }

    try:
        order = exchange.create_order(symbol, 'limit', binance_side.lower(), amount, round_price(price), params)
        # Track open orders locally
        open_orders.append({
            'id': order['id'],
            'grid_id': grid_id,
            'side': binance_side.lower(),
            'price': float(price),
            'size_usd': float(size_usd),
            'status': order['status'],
            'tp': float(tp),
            'sl': float(sl)
        })

        if grid_id:
            grids[grid_id]['orders'].append(order['id'])

        print(f"[GRID ORDER] {side.upper()} order placed at {price:.2f} (tp={tp:.2f}, sl={sl:.2f})")

    except Exception as e:
        print("[ERROR PLACING GRID ORDER]", e)


In [187]:
def place_grid(grid_direction, center_price, mode='trend', fraction=position_fraction):
    """
    Symmetric ladder grid system on Binance Futures:
    - 'up' creates buy levels (long entries)
    - 'down' creates sell levels (short entries)
    """
    gid = str(uuid.uuid4())[:8]
    grids[gid] = {
        'id': gid,
        'direction': grid_direction,
        'active': True,
        'created_at': datetime.now(timezone.utc),  # ✅ fixed deprecation warning
        'orders': [],
        'frozen': False
    }

    atr_now = get_current_atr()
    spacing = atr_now * grid_spacing_atr_factor
    size_usd = max(20.0, balance * fraction)

    half = grid_levels // 2
    grid_prices = [center_price + i * spacing for i in range(-half, half + 1)]
    grid_prices = sorted(set(grid_prices))

    # Determine Binance side
    binance_side = 'SELL' if grid_direction == 'down' else 'BUY'

    # ✅ Get Binance symbol info for precision and min limits
    market = exchange.market(symbol)
    min_qty = market['limits']['amount']['min'] or 0.0
    min_cost = market['limits']['cost']['min'] or 5.0  # typical minimum $5 notional
    amount_precision = int(abs(math.log10(market['precision']['amount'])))
    price_precision = int(abs(math.log10(market['precision']['price'])))


    for i, level_price in enumerate(grid_prices):
        # Round price
        price_rounded = round(level_price, price_precision)

        # Convert USD size → quantity
        qty = size_usd / price_rounded
        qty = round(qty, amount_precision)

        # ✅ Enforce Binance minimum rules
        if qty < min_qty or size_usd < min_cost:
            print(f"[SKIP GRID ORDER] qty={qty:.8f}, min={min_qty}, notional={size_usd:.2f} < {min_cost}")
            continue

        # Take Profit
        if i + 1 < len(grid_prices):
            tp_price = grid_prices[i + 1]
        else:
            tp_price = level_price * (1 + min_tp_pct if grid_direction == 'up' else 1 - min_tp_pct)

        # Stop Loss
        sl_price = level_price - 0.5 * spacing if grid_direction == 'up' else level_price + 0.5 * spacing

        # ✅ Place real Binance limit order
        try:
            resp = exchange.create_order(symbol, 'limit', binance_side.lower(), qty, price_rounded)
            ccxt_id = resp.get('id')
        except Exception as e:
            print("[ERROR PLACING GRID ORDER]", e)
            ccxt_id = None

        # Track order locally
        oid = str(uuid.uuid4())[:10]
        order = {
            'id': oid,
            'grid_id': gid,
            'side': binance_side.lower(),
            'price': float(price_rounded),
            'qty': float(qty),
            'size_usd': float(size_usd),
            'status': 'open' if ccxt_id else 'failed',
            'ccxt_id': ccxt_id,
            'tp': float(tp_price),
            'sl': float(sl_price)
        }
        open_orders.append(order)
        grids[gid]['orders'].append(oid)

    print(f"[GRID PLACED] id={gid}, dir={grid_direction}, mode={mode}, levels={len(grid_prices)}")
    print("------------------------------------------------------------")
    print(f"[DEBUG] Grid Details for {gid}:")
    for i, level_price in enumerate(grid_prices):
        if i + 1 < len(grid_prices):
            tp_price = grid_prices[i + 1]
        else:
            tp_price = level_price * (1 + min_tp_pct if grid_direction == 'up' else 1 - min_tp_pct)
        sl_price = level_price - 0.5 * spacing if grid_direction == 'up' else level_price + 0.5 * spacing
        print(f"Grid {i+1}/{len(grid_prices)} | Price: {level_price:.2f} | TP: {tp_price:.2f} | SL: {sl_price:.2f}")
    print("------------------------------------------------------------")
    return gid


In [188]:
# -------------------------
# Grid management functions for real Binance connection
# -------------------------
from datetime import datetime, timezone

def freeze_grid(gid):
    """Freeze a grid to stop creating new orders and deactivate it."""
    if gid in grids:
        grids[gid]['frozen'] = True
        grids[gid]['active'] = False
        grids[gid]['frozen_at'] = datetime.now(timezone.utc)
        print(f"[GRID FROZEN] id={gid}")


def cancel_grid_orders_ccxt(gid):
    """
    Cancel all open orders in Binance for a given grid and update local bookkeeping.
    """
    global open_orders
    # Filter local open orders for this grid
    to_cancel = [o for o in open_orders if o['grid_id'] == gid and o['status'] == 'open']

    for o in to_cancel:
        ccid = o.get('ccxt_id')
        if ccid:
            try:
                exchange.cancel_order(ccid, symbol)
                print(f"[ORDER CANCELLED] ccxt_id={ccid}, grid_id={gid}")
            except Exception as e:
                print(f"[CANCEL ERROR] ccxt_id={ccid}: {e}")
        # Update local status regardless
        o['status'] = 'cancelled'

    # Optionally remove the grid metadata
    grids.pop(gid, None)
    print(f"[GRID CANCELLED] id={gid}, cancelled_orders={len(to_cancel)}")


def get_open_orders_for_grid(gid):
    """Return all locally tracked open orders for a specific grid."""
    return [o for o in open_orders if o['grid_id'] == gid and o['status'] == 'open']


def total_open_exposure():
    """
    Calculate total USD exposure:
    - Positions: use 'notional' (from Binance positions)
    - Orders: sum of size_usd for open orders
    """
    pos_exposure = sum([p['notional'] for p in open_positions])
    orders_exposure = sum([o['size_usd'] for o in open_orders if o['status'] == 'open'])
    return pos_exposure + orders_exposure


def has_position_from_grid(gid):
    """Check if there is any active position opened from a grid."""
    return any(p.get('grid_id') == gid and p.get('active', True) for p in open_positions)


def get_next_higher_price(entry_price):
    """Find the next higher buy order price above entry_price in locally tracked open orders."""
    buy_prices = sorted([o['price'] for o in open_orders if o['status'] == 'open' and o['side'] == 'buy'])
    for p in buy_prices:
        if p > entry_price:
            return p
    return entry_price * (1 + min_tp_pct)


In [189]:
def try_update_order_statuses():
    """
    Poll Binance via CCXT for open orders and update their statuses in local open_orders.
    """
    try:
        # Fetch live open orders for the symbol
        live_open = exchange.fetch_open_orders(symbol)
        live_ids = {o['id'] for o in live_open}

        for o in open_orders:
            ccid = o.get('ccxt_id')
            if not ccid:
                continue  # skip orders that weren't sent to Binance
            if ccid not in live_ids and o['status'] == 'open':
                # Order no longer open: fetch details to get final status
                try:
                    det = exchange.fetch_order(ccid, symbol)
                    o['status'] = det.get('status', 'closed')
                    print(f"[ORDER UPDATE] ccxt_id={ccid}, status={o['status']}")
                except Exception as e:
                    print(f"[ORDER FETCH ERROR] ccxt_id={ccid}: {e}")

    except Exception as e:
        print("[CCXT POLL ERROR]", e)


In [190]:
def open_long(current_price, max_usd):
    """Open LONG orders in grid zones when price touches, placing real Binance orders."""
    global open_orders, open_positions, balance
    filled = []

    for o in list(open_orders):
        if o['status'] != 'open' or o['side'] != 'buy':
            continue

        grid_id = o['grid_id']

        # Calculate bounds
        grid_orders = [oo for oo in open_orders if oo['grid_id']==grid_id and oo['status']=='open' and oo['side']=='buy']
        grid_prices = sorted([oo['price'] for oo in grid_orders])
        i = grid_prices.index(o['price'])
        lower_bound = grid_prices[i - 1] if i > 0 else o['price'] * 0.995
        upper_bound = grid_prices[i + 1] if i + 1 < len(grid_prices) else o['price'] * 1.005

        # Skip if zone already occupied
        if any(p['grid_id'] == grid_id and lower_bound < p['entry'] <= upper_bound for p in open_positions):
            continue

        # Price touches zone
        if lower_bound < current_price <= o['price']:
            # Get available balance from Binance
            balance_data = exchange.fetch_balance()
            usdt_balance = balance_data['total'].get('USDT', 0)
            size_usd = min(max_usd, usdt_balance)
            if size_usd <= 0:
                print("❌ Not enough balance to open LONG.")
                continue

            # Place real limit order
            try:
                order_resp = exchange.create_order(
                    symbol=symbol,
                    type='LIMIT',
                    side='BUY',
                    amount=size_usd / o['price'],  # Binance expects quantity
                    price=o['price'],
                    params={'timeInForce': 'GTC'}
                )
                ccxt_id = order_resp['id']
                o['ccxt_id'] = ccxt_id
                print(f"[BINANCE LONG ORDER PLACED] id={ccxt_id} price={o['price']:.2f} size_usd={size_usd:.2f}")
            except Exception as e:
                print("[BINANCE ORDER ERROR]", e)
                continue

            # Record locally
            pos = {
                'id': str(uuid.uuid4())[:10],
                'side': 'long',
                'entry': float(o['price']),
                'grid_id': grid_id,
                'size_usd': size_usd,
                'ccxt_id': ccxt_id,
                'active': True,
                'tp_price': float(o.get('tp')),
                'sl_price': float(o.get('sl'))
            }
            open_positions.append(pos)
            o['status'] = 'placed'
            filled.append(o)

    return filled


def open_short(current_price, max_usd):
    """Open SHORT orders in grid zones when price touches, placing real Binance orders."""
    global open_orders, open_positions, balance
    filled = []

    for o in list(open_orders):
        if o['status'] != 'open' or o['side'] != 'sell':
            continue

        grid_id = o['grid_id']

        # Calculate bounds
        grid_orders = [oo for oo in open_orders if oo['grid_id']==grid_id and oo['status']=='open' and oo['side']=='sell']
        grid_prices = sorted([oo['price'] for oo in grid_orders])
        i = grid_prices.index(o['price'])
        lower_bound = grid_prices[i - 1] if i > 0 else o['price'] * 0.995
        upper_bound = grid_prices[i + 1] if i + 1 < len(grid_prices) else o['price'] * 1.005

        # Skip if zone already occupied
        if any(p['grid_id'] == grid_id and lower_bound < p['entry'] <= upper_bound for p in open_positions):
            continue

        # Price touches zone
        if upper_bound > current_price >= o['price']:
            # Get available balance from Binance
            balance_data = exchange.fetch_balance()
            usdt_balance = balance_data['total'].get('USDT', 0)
            size_usd = min(max_usd, usdt_balance)
            if size_usd <= 0:
                print("❌ Not enough balance to open SHORT.")
                continue

            # Place real limit order
            try:
                order_resp = exchange.create_order(
                    symbol=symbol,
                    type='LIMIT',
                    side='SELL',
                    amount=size_usd / o['price'],  # Binance expects quantity
                    price=o['price'],
                    params={'timeInForce': 'GTC'}
                )
                ccxt_id = order_resp['id']
                o['ccxt_id'] = ccxt_id
                print(f"[BINANCE SHORT ORDER PLACED] id={ccxt_id} price={o['price']:.2f} size_usd={size_usd:.2f}")
            except Exception as e:
                print("[BINANCE ORDER ERROR]", e)
                continue

            # Record locally
            pos = {
                'id': str(uuid.uuid4())[:10],
                'side': 'short',
                'entry': float(o['price']),
                'grid_id': grid_id,
                'size_usd': size_usd,
                'ccxt_id': ccxt_id,
                'active': True,
                'tp_price': float(o.get('tp')),
                'sl_price': float(o.get('sl'))
            }
            open_positions.append(pos)
            o['status'] = 'placed'
            filled.append(o)

    return filled


In [191]:
def try_resolve_positions_by_tp_sl(current_price):
    """
    Resolve positions by TP and SL using real Binance orders (robust).
    Works with mixed local positions and positions imported from exchange.
    Returns a list of closed positions info: (pos, reason, exit_price, pnl)
    """
    global open_positions
    closed = []

    for pos in list(open_positions):
        # defensive accessors
        grid_id = pos.get('grid_id')                  # may be None
        entry_price = pos.get('entry')                # may be None
        tp = pos.get('tp_price') or pos.get('tp')     # accept multiple keys
        sl = pos.get('sl_price') or pos.get('sl')
        qty = pos.get('qty')
        side = pos.get('side')
        ccxt_pos = pos.get('ccxt_pos')                # optional structure from exchange
        ccxt_id = pos.get('ccxt_id')                  # order id if stored

        # Attempt to derive missing fields from ccxt_pos if present
        if ccxt_pos:
            # ccxt_pos may contain 'contracts', 'entryPrice', 'amount', etc.
            try:
                if qty is None:
                    # many ccxt implementations: 'contracts' or 'amount'
                    qty = float(ccxt_pos.get('contracts') or ccxt_pos.get('amount') or 0) 
                if entry_price is None:
                    entry_price = float(ccxt_pos.get('entryPrice') or ccxt_pos.get('info', {}).get('entryPrice') or entry_price or 0)
            except Exception:
                pass

        # If still missing qty or side, try to derive from 'notional' and entry
        if qty is None:
            notional = pos.get('notional') or pos.get('size_usd') or 0
            if entry_price and notional:
                qty = notional / entry_price
            else:
                # nothing useful: skip this pos and log
                print(f"[RESOLVE SKIP] pos={pos.get('id')} missing qty/entry; pos keys={list(pos.keys())}")
                continue

        # ensure numeric types
        try:
            qty = float(qty)
            entry_price = float(entry_price)
        except Exception:
            print(f"[RESOLVE SKIP] pos={pos.get('id')} invalid qty/entry values.")
            continue

        try:
            # LONG position handling
            if side == 'long':
                # TAKE PROFIT
                if tp is not None and current_price >= float(tp):
                    # place market SELL reduceOnly
                    try:
                        resp = exchange.create_order(symbol, 'market', 'sell', qty, None, {'reduceOnly': True})
                        # Try to get an exit price (best-effort)
                        exit_price = None
                        if isinstance(resp, dict):
                            exit_price = float(resp.get('average') or resp.get('avgFillPrice') or resp.get('price') or current_price)
                        exit_price = exit_price or current_price
                    except Exception as e:
                        print("[BINANCE CLOSE ERROR LONG TP]", e)
                        exit_price = current_price

                    pnl = (exit_price - entry_price) * qty
                    open_positions.remove(pos)
                    closed.append((pos, 'TP', exit_price, pnl))
                    print(f"[POS TP] id={pos.get('id')} pnl={pnl:.2f} tp={exit_price:.2f} entry={entry_price:.2f} cur={current_price:.2f}")

                    # recreate grid order only if we know where it came from
                    if grid_id:
                        recreate_grid_order(entry_price, 'long')

                # STOP LOSS
                elif sl is not None and current_price <= float(sl):
                    try:
                        resp = exchange.create_order(symbol, 'market', 'sell', qty, None, {'reduceOnly': True})
                        exit_price = float(resp.get('average') or resp.get('avgFillPrice') or current_price)
                    except Exception as e:
                        print("[BINANCE CLOSE ERROR LONG SL]", e)
                        exit_price = current_price

                    pnl = (exit_price - entry_price) * qty
                    open_positions.remove(pos)
                    closed.append((pos, 'SL', exit_price, pnl))
                    print(f"[POS SL] id={pos.get('id')} pnl={pnl:.2f} sl={exit_price:.2f} entry={entry_price:.2f} cur={current_price:.2f}")

                    if grid_id:
                        recreate_grid_order(entry_price, 'long')

            # SHORT position handling
            elif side == 'short':
                # TAKE PROFIT (price dropped)
                if tp is not None and current_price <= float(tp):
                    try:
                        resp = exchange.create_order(symbol, 'market', 'buy', qty, None, {'reduceOnly': True})
                        exit_price = float(resp.get('average') or resp.get('avgFillPrice') or current_price)
                    except Exception as e:
                        print("[BINANCE CLOSE ERROR SHORT TP]", e)
                        exit_price = current_price

                    pnl = (entry_price - exit_price) * qty
                    open_positions.remove(pos)
                    closed.append((pos, 'TP_SHORT', exit_price, pnl))
                    print(f"[POS TP SHORT] id={pos.get('id')} pnl={pnl:.2f} tp={exit_price:.2f} entry={entry_price:.2f} cur={current_price:.2f}")

                    if grid_id:
                        recreate_grid_order(entry_price, 'short')

                # STOP LOSS (price rose)
                elif sl is not None and current_price >= float(sl):
                    try:
                        resp = exchange.create_order(symbol, 'market', 'buy', qty, None, {'reduceOnly': True})
                        exit_price = float(resp.get('average') or resp.get('avgFillPrice') or current_price)
                    except Exception as e:
                        print("[BINANCE CLOSE ERROR SHORT SL]", e)
                        exit_price = current_price

                    pnl = (entry_price - exit_price) * qty
                    open_positions.remove(pos)
                    closed.append((pos, 'SL_SHORT', exit_price, pnl))
                    print(f"[POS SL SHORT] id={pos.get('id')} pnl={pnl:.2f} sl={exit_price:.2f} entry={entry_price:.2f} cur={current_price:.2f}")

                    if grid_id:
                        recreate_grid_order(entry_price, 'short')

            else:
                # Unknown/unsupported side; skip
                print(f"[RESOLVE SKIP] pos={pos.get('id')} unsupported side={side}")
                continue

        except Exception as e:
            print("[BINANCE POSITION CLOSE ERROR]", e)
            # do not remove the pos here — leave it for next cycle

    return closed


In [192]:
# -------------------------
# Graceful transition & frozen cutoffs (Binance version)
# -------------------------

def begin_graceful_transition(new_prediction, center_price):
    """
    Freeze current grids and start a new grid according to prediction.
    """
    active_gids = [gid for gid, meta in grids.items() if not meta.get('frozen', False)]
    for gid in active_gids:
        freeze_grid(gid)

    any_open_in_frozen = any(
        len(get_open_orders_for_grid(gid)) > 0 or any(p['grid_id'] == gid for p in open_positions)
        for gid in active_gids
    )

    fraction = position_fraction * (reduced_fraction if any_open_in_frozen else 1.0)
    dir_map = {2: 'up', 0: 'down', 1: 'hold'}
    direction = dir_map.get(new_prediction, 'hold')

    new_gid = place_grid(direction, center_price, mode='trend', fraction=fraction)
    print(f"[TRANSITION] started new grid {new_gid} dir={direction} fraction={fraction:.3f}")
    return new_gid


def check_frozen_cutoffs(current_price):
    """
    Check frozen grids: cancel open orders and close positions if timeout or no exposure.
    """
    for gid, meta in list(grids.items()):
        if not meta.get('frozen', False):
            continue

        frozen_at = meta.get('frozen_at', datetime.now(timezone.utc))
        age = (datetime.now(timezone.utc) - frozen_at).total_seconds()

        orders = get_open_orders_for_grid(gid)
        exposure = sum([o.get('size_usd', 0) for o in orders])

        # Clear frozen grid if no exposure
        if exposure <= 0 and not any(p['grid_id'] == gid for p in open_positions):
            print(f"[CLEANUP] frozen grid {gid} cleared (no exposure).")
            grids.pop(gid, None)
            continue

        # Forced cutoff if grid has aged too long
        if age > frozen_timeout_seconds:
            print(f"[FORCED CUTOFF] grid {gid} timed out ({age:.0f}s) → closing all positions & cancelling orders.")

            # Cancel open Binance orders
            for o in list(open_orders):
                if o['grid_id'] == gid and o['status'] == 'open':
                    ccid = o.get('ccxt_id')
                    if ccid:
                        try:
                            cancel_order_ccxt(ccid)
                        except Exception as e:
                            print("[BINANCE CANCEL ERROR]", e)
                    o['status'] = 'cancelled'

            # Close all open positions via Binance market orders
            for p in list(open_positions):
                if p['grid_id'] == gid:
                    try:
                        side = 'SELL' if p['side'] == 'long' else 'BUY'
                        qty = p['qty']
                        order_resp = exchange.create_order(
                            symbol=symbol,
                            type='MARKET',
                            side=side,
                            amount=qty,
                            params={'reduceOnly': True}
                        )
                        exit_price = float(order_resp.get('avgFillPrice', current_price))
                        pnl = (exit_price - p['entry']) * qty if p['side'] == 'long' else (p['entry'] - exit_price) * qty
                        print(f"[FORCED CLOSE] pos={p['id']} grid={gid} side={p['side']} entry={p['entry']:.2f} exit={exit_price:.2f} pnl={pnl:.2f}")
                    except Exception as e:
                        print("[BINANCE CLOSE ERROR]", e)
                    finally:
                        open_positions.remove(p)

            grids.pop(gid, None)
            print(f"[GRID REMOVED] {gid} after forced cutoff.\n")


def close_all_positions(current_price):
    """
    Force close all positions and cancel all open orders across all grids.
    """
    for gid, meta in list(grids.items()):
        # Cancel all open orders in this grid
        for o in list(open_orders):
            if o['grid_id'] == gid and o['status'] == 'open':
                ccid = o.get('ccxt_id')
                if ccid:
                    try:
                        cancel_order_ccxt(ccid)
                    except Exception as e:
                        print("[BINANCE CANCEL ERROR]", e)
                o['status'] = 'cancelled'

        # Close all positions in this grid via market orders
        for p in list(open_positions):
            if p['grid_id'] == gid:
                try:
                    side = 'SELL' if p['side'] == 'long' else 'BUY'
                    qty = p['qty']
                    order_resp = exchange.create_order(
                        symbol=symbol,
                        type='MARKET',
                        side=side,
                        amount=qty,
                        params={'reduceOnly': True}
                    )
                    exit_price = float(order_resp.get('avgFillPrice', current_price))
                    pnl = (exit_price - p['entry']) * qty if p['side'] == 'long' else (p['entry'] - exit_price) * qty
                    print(f"[FORCED CLOSE] pos={p['id']} grid={gid} side={p['side']} entry={p['entry']:.2f} exit={exit_price:.2f} pnl={pnl:.2f}")
                except Exception as e:
                    print("[BINANCE CLOSE ERROR]", e)
                finally:
                    open_positions.remove(p)

        grids.pop(gid, None)
        print(f"[GRID REMOVED] {gid} after forced closure.\n")


In [193]:
balance_data = exchange.fetch_balance()
balance = float(balance_data['total'].get('USDT', 0))
initial_balance = balance

def compute_realistic_profit(current_price):
    simulated_balance = balance
    for pos in open_positions:
        qty = pos['qty']
        if pos['side'] == 'long':
            pnl = (current_price - pos['entry']) * qty
        else:
            pnl = (pos['entry'] - current_price) * qty
        simulated_balance += pnl + pos.get('size_usd', 0)
    net_profit = simulated_balance - initial_balance
    return simulated_balance, net_profit


In [194]:
import time
from datetime import datetime, timezone, timedelta

# Helper: sync local open_orders from Binance (updates statuses and ccxt_id presence)
def sync_open_orders_from_binance():
    """
    Fetch open orders from Binance and update local open_orders list.
    - For orders returned by Binance, ensure we have a local entry (by ccxt_id).
    - For local orders missing on Binance (no ccxt_id), keep them as-is.
    """
    try:
        live_open = exchange.fetch_open_orders(symbol)  # list of dicts
    except Exception as e:
        print("[SYNC OPEN ORDERS ERROR]", e)
        return

    live_by_id = {o['id']: o for o in live_open}

    # mark locally tracked orders that are no longer open on Binance as 'filled'/'closed' if appropriate
    for o in open_orders:
        ccid = o.get('ccxt_id')
        if not ccid:
            # local-only order (placed but no ccxt id) — leave it
            continue
        if ccid in live_by_id:
            # update local status with Binance status for live orders
            o['status'] = live_by_id[ccid].get('status', 'open')
            # update amount/price info if changed
            o['price'] = float(live_by_id[ccid].get('price', o['price']))
            o['exchange_info'] = live_by_id[ccid]
        else:
            # not in live open orders -> might be filled or canceled; fetch details
            try:
                det = exchange.fetch_order(ccid, symbol)
                o['status'] = det.get('status', 'closed')
                o['exchange_info'] = det
            except Exception as e:
                # could be order not found or API error; mark as unknown but don't remove
                print(f"[SYNC ORDER DETAIL ERROR] ccid={ccid}: {e}")

    # Add any live orders from Binance that we don't have locally (helpful after restart)
    for live_id, live in live_by_id.items():
        if not any((o.get('ccxt_id') == live_id) for o in open_orders):
            # create a local record to track it (best-effort)
            local_oid = str(uuid.uuid4())[:10]
            local_order = {
                'id': local_oid,
                'grid_id': None,
                'side': live.get('side', '').lower(),
                'price': float(live.get('price', 0) or 0),
                'size_usd': None,
                'status': live.get('status', 'open'),
                'ccxt_id': live_id,
                'tp': None,
                'sl': None,
                'exchange_info': live
            }
            open_orders.append(local_order)
            print(f"[SYNC] Imported live open order into local store: ccid={live_id}")

# Helper: sync local open_positions from Binance positions
def sync_positions_from_binance():
    """
    Fetch positions via ccxt and update local open_positions.
    This maps exchange position fields into your bot's format (id, side, entry, qty, notional, grid_id, ccxt_pos).
    """
    global open_positions
    try:
        positions = exchange.fetch_positions()  # standard ccxt method for futures
    except Exception as e:
        print("[SYNC POSITIONS ERROR]", e)
        return

    # Clear local open_positions and rebuild from Binance positions to avoid duplicates
    updated = []
    for p in positions:
        # Some exchanges return many positions; filter by symbol
        if p.get('symbol') != symbol:
            continue
        contracts = float(p.get('contracts') or 0)
        if abs(contracts) <= 0:
            continue  # skip zero positions
        entry = float(p.get('entryPrice') or p.get('info', {}).get('entryPrice') or 0)
        notional = float(p.get('notional') or (abs(contracts) * entry) or 0)
        side = 'long' if contracts > 0 else 'short'
        qty = abs(contracts)
        local_pos = {
            'id': str(uuid.uuid4())[:10],
            'side': side,
            'entry': entry,
            'qty': qty,
            'notional': notional,
            'grid_id': None,               # we may not know grid origin without storing it elsewhere
            'created_at': datetime.now(timezone.utc),
            'last_price': float(p.get('markPrice') or 0),
            'active': True,
            'tp_price': None,
            'sl_price': None,
            'ccxt_pos': p
        }
        updated.append(local_pos)

    if updated:
        # replace only if we found active positions (so we don't wipe local tracker on API failure)
        open_positions = updated

# -------------------------
# Main run loop (Binance Testnet)
# -------------------------
def run_bot():
    """
    Main loop for running the bot connected to Binance Futures Testnet.
    Key points:
      - Uses exchange.fetch_balance() for real USDT balance.
      - Uses exchange.fetch_ticker() for live price.
      - Syncs open orders & positions with Binance regularly.
      - Places grid orders with place_grid() that call place_limit_order_ccxt (which should create real CCXT orders).
      - Closes positions with try_resolve_positions_by_tp_sl (which should place market reduceOnly orders).
    """
    wait_for_reversal = False
    global balance, open_positions, open_orders, grids

    last_prediction = None
    last_prediction_time = None
    last_trend_direction = None

    # Warm-up (model + last reversal)
    back_df = fetch_latest_ohlcv(symbol, model_timeframe, 120 + window_size)
    feat_df = compute_indicators(back_df).dropna()
    preds = [
        predict_label_from_window(feat_df[FEATURES].values[i - window_size:i])
        for i in range(window_size, len(feat_df))
    ]
    last_reversal = next((p for p in reversed(preds) if p != 1), None)
    last_prediction = last_reversal if last_reversal is not None else 1
    last_prediction_time = datetime.now(timezone.utc)

    # Initial sync with Binance so local bookkeeping starts accurate
    try:
        balance_data = exchange.fetch_balance()
        balance = float(balance_data['total'].get('USDT', 0))
    except Exception as e:
        print("[INIT BALANCE ERROR]", e)
        balance = 0.0

    # sync live orders & positions (important after restart)
    sync_open_orders_from_binance()
    sync_positions_from_binance()

    # fetch a live price
    try:
        last_price = float(exchange.fetch_ticker(symbol)['last'])
    except Exception:
        last_price = float(fetch_latest_ohlcv_2(symbol, price_timeframe, 2)['close'].values[-1])

    print(f"[INIT] last_pred={last_prediction} price={last_price:.2f} usdt_balance={balance:.2f}")

    # Place initial grid based on last reversal
    if last_prediction == 0:
        place_grid('down', last_price, mode='trend')
        last_trend_direction = 'short'
    elif last_prediction == 2:
        place_grid('up', last_price, mode='trend')
        last_trend_direction = 'long'
    else:
        print("[INIT] No clear trend — waiting for prediction.")

    try:
        while True:
            now = datetime.now(timezone.utc)
            do_predict = (last_prediction_time is None) or ((now - last_prediction_time).total_seconds() >= prediction_interval)

            # Update live balance & sync before decisions
            try:
                balance_data = exchange.fetch_balance()
                balance = float(balance_data['total'].get('USDT', 0))
            except Exception as e:
                print("[BALANCE FETCH ERROR]", e)

            # Periodically sync open orders and positions (keeps local state in-line with Binance)
            sync_open_orders_from_binance()
            sync_positions_from_binance()

            if do_predict:
                # --- Model prediction step (unchanged) ---
                df_feat = fetch_latest_ohlcv(symbol, model_timeframe, limit)
                feat_df = compute_indicators(df_feat).dropna()
                if len(feat_df) >= window_size:
                    window = feat_df[FEATURES].values[-window_size:]
                    new_pred = predict_label_from_window(window)
                    print(f"[{datetime.now(timezone.utc)}] ML pred={new_pred}")
                else:
                    new_pred = last_prediction

                # act on prediction changes
                if new_pred != last_prediction:
                    center_price = float(exchange.fetch_ticker(symbol)['last'])
                    # DOWN -> close and start short grid
                    if new_pred == 0:
                        print("[PRED] DOWN -> closing and starting SHORT grid")
                        close_all_positions(center_price)
                        for gid in list(grids.keys()):
                            cancel_grid_orders_ccxt(gid)
                            grids.pop(gid, None)
                        place_grid('down', center_price, mode='trend')
                        last_trend_direction = 'short'
                        last_prediction = 0

                    # UP -> close and start long grid
                    elif new_pred == 2:
                        print("[PRED] UP -> switching to LONG grid")
                        close_all_positions(center_price)
                        for gid in list(grids.keys()):
                            cancel_grid_orders_ccxt(gid)
                            grids.pop(gid, None)
                        place_grid('up', center_price, mode='trend')
                        last_trend_direction = 'long'
                        last_prediction = 2

                    # HOLD: maintain/adjust depending on last trend
                    elif new_pred == 1:
                        print("[PRED] HOLD -> keep/refresh grids")
                        # keep last_trend_direction but refresh grid if needed
                        center_price = float(exchange.fetch_ticker(symbol)['last'])
                        close_all_positions(center_price)
                        for gid in list(grids.keys()):
                            cancel_grid_orders_ccxt(gid)
                            grids.pop(gid, None)
                        if last_trend_direction == 'short':
                            place_grid('down', center_price, mode='trend')
                        else:
                            place_grid('up', center_price, mode='trend')
                        last_prediction = 1

                else:
                    # prediction unchanged
                    pass

                last_prediction_time = datetime.now(timezone.utc)

            # --- Inner loop: run until next prediction time ---
            inner_deadline = datetime.now(timezone.utc) + timedelta(seconds=prediction_interval)
            while datetime.now(timezone.utc) < inner_deadline:
                # Live price
                try:
                    last_price = float(exchange.fetch_ticker(symbol)['last'])
                except Exception:
                    # fallback
                    last_price = float(fetch_latest_ohlcv_2(symbol, price_timeframe, 2)['close'].values[-1])

                # update indicators & model if required (quick check)
                feat_df = compute_indicators(fetch_latest_ohlcv(symbol, model_timeframe, limit)).dropna()
                if len(feat_df) >= window_size:
                    window = feat_df[FEATURES].values[-window_size:]
                    current_pred = predict_label_from_window(window)
                else:
                    current_pred = last_prediction

                # WAIT MODE handling (same logic, but reporting uses real balance)
                if wait_for_reversal:
                    if (current_pred == 0 and last_trend_direction != 'short') or (current_pred == 2 and last_trend_direction != 'long'):
                        print("[REVERSAL DETECTED] exit WAIT MODE")
                        wait_for_reversal = False
                        # reset and place grid for new direction
                        for gid in list(grids.keys()):
                            cancel_grid_orders_ccxt(gid)
                            grids.pop(gid, None)
                        if current_pred == 0:
                            place_grid('down', last_price, mode='trend')
                            last_trend_direction = 'short'
                        elif current_pred == 2:
                            place_grid('up', last_price, mode='trend')
                            last_trend_direction = 'long'
                        last_prediction = current_pred
                    else:
                        # compute current_sim profit only for reporting (optional)
                        simulated_balance, profit_now = compute_realistic_profit(last_price)
                        print(f"[WAIT MODE] balance={balance:.2f}, sim_bal={simulated_balance:.2f}, profit={profit_now:.2f}")
                        time.sleep(check_interval_seconds)
                        continue

                # Dynamic switching based on inner prediction
                if current_pred == 0 and last_trend_direction == 'long':
                    print("[INNER] switch to SHORT grid")
                    close_all_positions(last_price)
                    for gid in list(grids.keys()):
                        cancel_grid_orders_ccxt(gid)
                        grids.pop(gid, None)
                    place_grid('down', last_price, mode='trend')
                    last_trend_direction = 'short'
                    last_prediction = 0

                elif current_pred == 2 and last_trend_direction == 'short':
                    print("[INNER] switch to LONG grid")
                    close_all_positions(last_price)
                    for gid in list(grids.keys()):
                        cancel_grid_orders_ccxt(gid)
                        grids.pop(gid, None)
                    place_grid('up', last_price, mode='trend')
                    last_trend_direction = 'long'
                    last_prediction = 2

                # System maintenance: update orders & resolve TP/SL with real Binance actions
                try_update_order_statuses()   # updates local open_orders using exchange data
                closed = try_resolve_positions_by_tp_sl(last_price)
                if closed:
                    print(f"[RESOLVE] closed {len(closed)} positions")

                # Trading activity — determine how much to allocate (use real balance)
                trade_usd = max(20.0, balance * position_fraction) if balance > 0 else 0

                if last_prediction == 2:
                    open_long(last_price, trade_usd)
                elif last_prediction == 0:
                    open_short(last_price, trade_usd)
                elif current_pred == 1 and last_trend_direction == 'long':
                    open_long(last_price, trade_usd)
                elif current_pred == 1 and last_trend_direction == 'short':
                    open_short(last_price, trade_usd)

                # housekeeping
                check_frozen_cutoffs(last_price)

                # grid cleanup
                for gid, meta in list(grids.items()):
                    if not meta.get('frozen', False):
                        orders = get_open_orders_for_grid(gid)
                        has_pos = any(p['grid_id'] == gid for p in open_positions)
                        if len(orders) == 0 and not has_pos:
                            print(f"[CLEANUP] grid {gid} has no exposure, removing")
                            grids.pop(gid, None)

                # Exposure & safety: compute exposure vs real balance
                exposure = total_open_exposure()
                exposure_pct = exposure / max(1e-8, balance)
                allow_place = exposure_pct <= max_total_exposure
                active_dirs = [meta['direction'] for gid, meta in grids.items() if not meta.get('frozen', False)]

                # place missing grid if required
                desired_dir = None
                if last_prediction == 2:
                    desired_dir = 'up'
                elif last_prediction == 0:
                    desired_dir = 'down'
                elif current_pred == 1 and last_trend_direction == 'long':
                    desired_dir = 'up'
                elif current_pred == 1 and last_trend_direction == 'short':
                    desired_dir = 'down'

                if desired_dir and desired_dir not in active_dirs and allow_place:
                    place_grid(desired_dir, last_price, mode='trend')

                # reporting: compute local simulated profit for decision-making (optional)
                simulated_balance, profit_now = compute_realistic_profit(last_price)
                print(f"[STATUS] price={last_price:.2f} balance={balance:.2f} sim_if_stopped={simulated_balance:.2f} profit={profit_now:.2f} exposure={exposure:.2f} grids={len(grids)}")

                # drawdown protection (using simulated profit or alternative threshold)
                if profit_now <= -1.20:
                    print("[DRAWDOWN] severe loss → closing everything and waiting for reversal")
                    close_all_positions(last_price)
                    for gid in list(grids.keys()):
                        cancel_grid_orders_ccxt(gid)
                        grids.pop(gid, None)
                    wait_for_reversal = True
                    break

                time.sleep(check_interval_seconds)

    except KeyboardInterrupt:
        print("User stopped the bot.")
        try:
            bal_data = exchange.fetch_balance()
            print("Live USDT balance:", bal_data['total'].get('USDT'))
        except Exception:
            pass
        print("Local open positions:", open_positions)
        print("Local open orders (open):", [o for o in open_orders if o['status'] in ('open','placed')])


In [195]:
# -------------------------
# Run
# -------------------------
if __name__ == "__main__":
    run_bot()

[SYNC] Imported live open order into local store: ccid=6114078692
[SYNC] Imported live open order into local store: ccid=6114079069
[SYNC] Imported live open order into local store: ccid=6114080178
[SYNC] Imported live open order into local store: ccid=6114081598
[SYNC] Imported live open order into local store: ccid=6114082109
[SYNC] Imported live open order into local store: ccid=6114083334
[SYNC] Imported live open order into local store: ccid=6114401910
[SYNC] Imported live open order into local store: ccid=6114403273
[SYNC] Imported live open order into local store: ccid=6114403897
[SYNC] Imported live open order into local store: ccid=6114405461
[SYNC] Imported live open order into local store: ccid=6114406010
[INIT] last_pred=2 price=105798.50 usdt_balance=4995.30
[GRID PLACED] id=157f982b, dir=up, mode=trend, levels=17
------------------------------------------------------------
[DEBUG] Grid Details for 157f982b:
Grid 1/17 | Price: 105318.52 | TP: 105378.52 | SL: 105288.52
Grid

KeyError: 'grid_id'